Autor: so jae woo    
Data src :  qesg news / 28 cat / 본문 포함.    
Summary : KLUE기반의 버트 사정 임베딩 모델 사용.   


In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import transformers
transformers.__version__ # 
from sklearn.model_selection import StratifiedKFold

from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import TextClassificationPipeline
from sklearn.utils import class_weight

from lib_offline import prep_mecab_noun_offline
from lib2 import text_preprocessing
import pickle

# 데이터 준비 
## QESG 데이터 준비 

In [2]:
df_qesg = pd.read_parquet("./df_qesg28_final.pq")
df_qesg = df_qesg.loc[df_qesg.y.notna()]
df_qesg = df_qesg.loc[df_qesg.y != '기타']
df_qesg = df_qesg.loc[df_qesg.y != '제외']

In [15]:
len(pd.unique(df_qesg.y))

34

### 데이터 클리닝

### 짧은 본문 제거

In [3]:
def check_contents_clean(row):
    c = row.contents
    cc = row.contents_clean
    
    if len(c) == 0:
        return False
    
    if len(cc)/len(c)> 0.7:
        return True
    else:
        return False

In [4]:

bol_list = [ check_contents_clean(row) for i , row in df_qesg.iterrows() ]
df_qesg['size'] = df_qesg[bol_list].contents_clean.apply(lambda x : len(x))
df_qesg = df_qesg.loc[df_qesg['size'] > 40]

# BERT 임베딩 

In [5]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [6]:
# load tokenizer 
max_seq_len = 2028
tokenizer = BertTokenizer.from_pretrained('klue/bert-base', truncation=True, max_seq_len=max_seq_len)

In [7]:
## 라벨 숫자 인코딩
lbl_name = sorted(df_qesg['y'].unique().tolist())
lbl_num = list(range(len(lbl_name)))
lbl_name2num = dict(zip(lbl_name,lbl_num))
lbl_num2name = dict(zip(lbl_num,lbl_name))
df_qesg['y2num']  = df_qesg.y.apply(lambda x : lbl_name2num[x])

# 트레이닝

In [8]:
X = np.array(df_qesg['contents_clean'].tolist())
y = np.array(df_qesg['y2num'].tolist())

In [9]:
# 원큐
class_weights = class_weight.compute_class_weight(class_weight  = 'balanced',classes= np.unique(y), y = y) 
class_weights_dict = dict(zip(  list(range(len(class_weights))),class_weights))

X_train = tokenizer(X.tolist(), truncation=True, padding=True)


train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y
))

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=df_qesg['y2num'].nunique(), from_pt=True)
loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history =model.fit(
train_dataset.shuffle(1000).batch(16), epochs=7, batch_size=32,class_weight= class_weights_dict,)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/7
3085/3085 [==============================] - 1298s 418ms/step - loss: 2.8413 - accuracy: 0.3397
Epoch 2/7
3085/3085 [==============================] - 1284s 416ms/step - loss: 3.5741 - accuracy: 0.0375
Epoch 3/7
3085/3085 [==============================] - 1284s 416ms/step - loss: 3.5500 - accuracy: 0.0391
Epoch 4/7
3085/3085 [==============================] - 1284s 416ms/step - loss: 3.4769 - accuracy: 0.0409
Epoch 5/7
3085/3085 [==============================] - 1284s 416ms/step - loss: 3.4344 - accuracy: 0.0344
Epoch 6/7
3085/3085 [==============================] - 1283s 416ms/step - loss: 3.4844 - accuracy: 0.0367
Epoch 7/7
3085/3085 [==============================] - 1283s 416ms/step - loss: 3.4380 - accuracy: 0.0337
